# Maze Robot Hybrid GA-PPO Approach

This notebook implements a hybrid approach that combines Genetic Algorithm (GA) with Proximal Policy Optimization (PPO). 
The key idea is to first use GA to evolve neural network architectures, then apply PPO to fine-tune the best policies.

In [1]:
# Import standard libraries
import numpy as np
import matplotlib.pyplot as plt
import pygame
import os
import cv2
import torch
import time
import math
from IPython.display import Video

# Import project modules
from Robot_config import Robot
from Environment import Environment
from class_GA import Genetic_Algo
from ppo_agent import PPO, ActorCritic
from reward_model import RewardModel
from ClassNeuralNetwork import NeuralNet
from save_metrics import save_ga_metrics, save_ppo_metrics

pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Setup headless pygame
os.environ["SDL_VIDEODRIVER"] = "dummy"
pygame.init()

# Configure video recording
def setup_video_writer(filename, fps=30, size=(900, 900)):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    return cv2.VideoWriter(filename, fourcc, fps, size)

# Helper function to convert pygame surface to numpy array for OpenCV
def surface_to_array(surface):
    return np.transpose(np.array(pygame.surfarray.pixels3d(surface)), (1, 0, 2))

## Hybrid GA-PPO Approach

Our hybrid approach works as follows:

1. Use GA to evolve a population of neural networks (similar to the original GA solution)
2. Take the best N individuals from the GA population
3. Convert each individual to an Actor-Critic architecture for PPO
4. Apply PPO to fine-tune each of these networks
5. Select the best performing network after PPO fine-tuning

In [3]:
# Function to convert a GA individual (neural network weights) to a PPO actor-critic model
def convert_ga_to_ppo_model(decoded_individual, state_dim=11, action_dim=2, hidden_dim=128):
    """
    Convert GA individual to PPO actor-critic model
    
    Parameters:
    -----------
    decoded_individual : array
        The decoded GA individual containing neural network weights
    """
    # Create a new PPO actor-critic model
    actor_critic = ActorCritic(state_dim, action_dim, hidden_dim)
    
    # Since GA uses a simple neural network and PPO uses a more complex architecture,
    # we'll initialize the feature extractor using weights from the GA individual
    w = np.reshape(decoded_individual[:110], (11, 10))  # First 110 elements are input->hidden weights
    v = np.reshape(decoded_individual[110:], (10, 2))   # Remaining elements are hidden->output weights
    
    # Set the first layer of feature extractor with GA weights
    with torch.no_grad():
        # Convert numpy arrays to PyTorch tensors
        w_tensor = torch.FloatTensor(w.T)  # Transpose for PyTorch's expected dimensions
        
        # Copy weights to PPO model's feature extractor first layer (with size adjustment)
        actor_critic.feature_extractor[0].weight.data[:10, :11] = w_tensor
        
        # Set output layer of actor with GA weights
        v_tensor = torch.FloatTensor(v.T)  # Transpose for PyTorch's expected dimensions
        actor_critic.actor_mean.weight.data[:2, :10] = v_tensor
    
    return actor_critic

In [4]:
def run_hybrid_ga_ppo(ga_generations=15, ppo_episodes=20, population_size=30, 
                     top_n_individuals=3, video_filename="hybrid_simulation.mp4"):
    print("Starting Hybrid GA-PPO Simulation")
    
    # Setup environment
    map = pygame.image.load("./maze.jpg")
    map_copy = map.copy()
    start_position = (450, 100)
    end_position = (450, 900, -np.pi/2)
    size = (900, 900)
    environment = Environment(size, "./maze.jpg")
    
    # Video writer
    video_writer = setup_video_writer(video_filename)
    
    # Step 1: Run GA to evolve initial population
    print("Phase 1: Running Genetic Algorithm...")
    
    # Setup robots
    Robots = [Robot(start_position, "./car.png", 2, 2) for _ in range(population_size)]
    
    # GA initialization
    GA_init = Genetic_Algo([-5, 5], (-5, 5), population_size, 130, 12)
    population = GA_init.create_population()
    
    # Metrics to track
    min_fitness_history = []
    mean_fitness_history = []
    best_distance_history = []
    best_individuals = []
    
    # GA loop
    for generation in range(ga_generations):
        print(f"GA Generation {generation+1}/{ga_generations}")
        
        # Reset robots
        for robot in Robots:
            robot.reset(start_position)
            robot.update_sensor_data(map_copy, (0, 0, 0))
        
        # Decode population
        decode = GA_init.decode_gen(population)
        
        # Evaluate population
        Robot_available = population_size
        dt = 0.05  # Fixed dt for simulation stability
        
        while Robot_available > 0:
            environment.map.blit(map, (0, 0))
            
            for idx, robot in enumerate(Robots):
                if not robot.crash:
                    # Neural network control from GA
                    decode_individual = decode[idx]
                    w = np.reshape(decode_individual[:110], (11, 10))
                    v = np.reshape(decode_individual[110:], (10, 2))
                    
                    x = np.array(robot.sensor_data + [float(robot.x), float(robot.y), float(robot.theta)]).reshape(11, 1)
                    
                    # Get control actions from neural network
                    VVV = NeuralNet(x, w, v).FeedForward()
                    robot.vx = (VVV[0][0]) * 2.5
                    robot.vtheta = (VVV[1][0]) * 0.03
                    
                    # Move robot
                    robot.move(dt)
                    robot.time += dt
                    robot.check_crash(map_copy, (0, 0, 0))
                    
                    # Record the end state if crashed
                    if robot.crash or robot.time > 30:
                        Robot_available -= 1
                    
                    # Update sensors
                    robot.update_sensor_data(map_copy, (0, 0, 0))
            
            # Record video only for certain frames and generations
            if generation in [0, ga_generations//2, ga_generations-1]:
                # Visualize
                environment.map.blit(map, (0, 0))
                for robot in Robots:
                    if not robot.crash:
                        robot.draw(environment.map)
                        environment.robot_frames([robot.x, robot.y], robot.theta)
                
                # Add generation info
                font = pygame.font.Font('freesansbold.ttf', 24)
                text = font.render(f"GA Generation: {generation+1}/{ga_generations}", True, (255, 255, 255), (0, 0, 0))
                environment.map.blit(text, (10, 10))
                
                # Record frame
                frame = surface_to_array(environment.map)
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                video_writer.write(frame)
        
        # Calculate fitness (distance to goal)
        fitness = []
        for robot in Robots:
            dist_to_goal = math.sqrt((robot.x - end_position[0])**2 + (robot.y - end_position[1])**2)
            fitness.append(dist_to_goal)
        
        # Record metrics
        min_fitness = np.min(fitness)
        mean_fitness = np.mean(fitness)
        min_fitness_history.append(min_fitness)
        mean_fitness_history.append(mean_fitness)
        best_distance_history.append(min_fitness)
        
        # Sort individuals by fitness (distance)
        sorted_indices = np.argsort(fitness)
        
        # Store the top N individuals
        best_individuals = [decode[i] for i in sorted_indices[:top_n_individuals]]
        
        # GA operations for next generation
        select_population = GA_init.selection(fitness, population)
        cross_population = GA_init.crossover(select_population)
        population = GA_init.mutation(cross_population, 0.2)
        
        print(f"Generation {generation+1} completed. Best distance: {min_fitness:.2f}, Mean: {mean_fitness:.2f}")
    
    # Save GA metrics
    save_ga_metrics(min_fitness_history, mean_fitness_history, best_distance_history, 'hybrid_ga_metrics.npz')
    
    # Step 2: Convert best GA individuals to PPO models and fine-tune
    print("\nPhase 2: PPO Fine-tuning of Best Individuals...")
    
    # Setup reward model for PPO
    reward_model = RewardModel(target_position=end_position[:2], target_orientation=end_position[2])
    
    best_ppo_rewards = []
    best_ppo_distances = []
    best_model = None
    best_reward = -float('inf')
    
    # For each of the top N individuals
    for individual_idx, individual in enumerate(best_individuals):
        print(f"\nFine-tuning Individual {individual_idx+1}/{len(best_individuals)}")
        
        # Convert GA individual to PPO model
        actor_critic = convert_ga_to_ppo_model(individual)
        
        # Create PPO agent with this model
        ppo_agent = PPO(state_dim=11, action_dim=2, hidden_dim=128)
        ppo_agent.actor_critic = actor_critic
        
        # Track metrics for this individual
        individual_rewards = []
        individual_distances = []
        
        # PPO fine-tuning loop
        for episode in range(ppo_episodes):
            # Reset robot
            robot = Robot(start_position, "./car.png", 2, 2)
            reward_model.reset((robot.x, robot.y))
            
            # Initial state
            robot.update_sensor_data(map_copy, (0, 0, 0))
            state = robot.get_state()
            
            episode_reward = 0
            done = False
            step = 0
            max_steps = 500
            
            while not done and step < max_steps:
                # Get action from policy
                action, log_prob = ppo_agent.select_action(state)
                
                # Execute action using robot's kinematic model
                robot.vx = action[0] * 2.5  # Scale linear velocity
                robot.vtheta = action[1] * 0.03  # Scale angular velocity
                robot.move(dt)
                robot.time += dt
                robot.update_sensor_data(map_copy, (0, 0, 0))
                robot.check_crash(map_copy, (0, 0, 0))
                
                # Get new state
                next_state = robot.get_state()
                
                # Calculate reward
                robot_state = {
                    'position': (robot.x, robot.y),
                    'orientation': robot.theta,
                    'velocity': (robot.x_dot, robot.y_dot),
                    'sensor_data': robot.sensor_data,
                    'crash': robot.crash,
                    'time': robot.time
                }
                reward, done = reward_model.calculate_reward(robot_state)
                
                # Store transition
                ppo_agent.store_transition(state, action, reward, next_state, done, log_prob)
                
                # Update state and reward
                state = next_state
                episode_reward += reward
                step += 1
                
                # Visualize and record at end of episode
                if done or step == max_steps-1:
                    if episode in [0, ppo_episodes//2, ppo_episodes-1]:
                        environment.map.blit(map, (0, 0))
                        robot.draw(environment.map)
                        environment.robot_frames([robot.x, robot.y], robot.theta)
                        environment.robot_sensor([robot.x, robot.y], robot.points)
                        
                        # Add info text
                        font = pygame.font.Font('freesansbold.ttf', 24)
                        text = font.render(f"PPO Individual {individual_idx+1}, Episode {episode+1}", True, (255, 255, 255), (0, 0, 0))
                        environment.map.blit(text, (10, 10))
                        
                        # Record frame
                        frame = surface_to_array(environment.map)
                        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                        video_writer.write(frame)
            
            # Update policy
            if (episode + 1) % 5 == 0:
                ppo_agent.update()
            
            # Calculate distance to goal
            distance = math.sqrt((robot.x - end_position[0])**2 + (robot.y - end_position[1])**2)
            
            # Store metrics
            individual_rewards.append(episode_reward)
            individual_distances.append(distance)
            
            print(f"Individual {individual_idx+1}, Episode {episode+1}: Reward = {episode_reward:.2f}, Distance = {distance:.2f}")
        
        # After training this individual, check if it's the best so far
        final_reward = np.mean(individual_rewards[-5:])  # Average of last 5 episodes
        if final_reward > best_reward:
            best_reward = final_reward
            best_model = ppo_agent.actor_critic
            best_ppo_rewards = individual_rewards
            best_ppo_distances = individual_distances
            # Save this model
            torch.save(ppo_agent.actor_critic.state_dict(), f"hybrid_best_individual_{individual_idx+1}.pt")
    
    # Save final best model
    if best_model is not None:
        torch.save(best_model.state_dict(), "hybrid_best_model.pt")
    
    # Save PPO metrics for the best individual
    save_ppo_metrics(best_ppo_rewards, best_ppo_distances, 'hybrid_ppo_metrics.npz')
    
    # Close video writer
    video_writer.release()
    
    print("\nHybrid GA-PPO Simulation Complete!")
    return min_fitness_history, mean_fitness_history, best_ppo_rewards, best_ppo_distances, video_filename

In [ ]:
# Run hybrid simulation with fewer generations/episodes for demonstration
ga_fitness_min, ga_fitness_mean, ppo_rewards, ppo_distances, video_file = run_hybrid_ga_ppo(
    ga_generations=10,  # Reduced for demonstration
    ppo_episodes=100,    # Reduced for demonstration
    population_size=200, # Reduced for demonstration
    top_n_individuals=5
)

Starting Hybrid GA-PPO Simulation
Phase 1: Running Genetic Algorithm...
GA Generation 1/10
Generation 1 completed. Best distance: 723.89, Mean: 814.31
GA Generation 2/10
Generation 2 completed. Best distance: 724.01, Mean: 802.93
GA Generation 3/10
Generation 3 completed. Best distance: 723.15, Mean: 796.04
GA Generation 4/10
Generation 4 completed. Best distance: 723.52, Mean: 792.01
GA Generation 5/10
Generation 5 completed. Best distance: 723.34, Mean: 776.11
GA Generation 6/10
Generation 6 completed. Best distance: 723.38, Mean: 783.27
GA Generation 7/10
Generation 7 completed. Best distance: 723.00, Mean: 787.06
GA Generation 8/10
Generation 8 completed. Best distance: 722.38, Mean: 773.05
GA Generation 9/10
Generation 9 completed. Best distance: 722.99, Mean: 783.65
GA Generation 10/10


In [ ]:
# Plot metrics
plt.figure(figsize=(15, 10))

# GA phase metrics
plt.subplot(2, 2, 1)
plt.plot(range(1, len(ga_fitness_min) + 1), ga_fitness_min, 'b-', label='Best Distance')
plt.plot(range(1, len(ga_fitness_mean) + 1), ga_fitness_mean, 'r-', label='Mean Distance')
plt.xlabel('Generation')
plt.ylabel('Distance to Goal (pixels)')
plt.title('GA Phase: Distance Evolution')
plt.legend()
plt.grid(True)

# PPO phase metrics
plt.subplot(2, 2, 2)
plt.plot(range(1, len(ppo_rewards) + 1), ppo_rewards, 'g-')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('PPO Phase: Reward Evolution')
plt.grid(True)

plt.subplot(2, 2, 3)
plt.plot(range(1, len(ppo_distances) + 1), ppo_distances, 'm-')
plt.xlabel('Episode')
plt.ylabel('Distance to Goal (pixels)')
plt.title('PPO Phase: Distance Evolution')
plt.grid(True)

# Combined plot - compare final GA distance with PPO distance
plt.subplot(2, 2, 4)
plt.bar(['GA Initial', 'GA Final', 'PPO Final'], 
       [ga_fitness_mean[0], ga_fitness_min[-1], ppo_distances[-1]], 
       color=['blue', 'green', 'purple'])
plt.ylabel('Distance to Goal (pixels)')
plt.title('Comparison: Initial vs Final Distances')

plt.tight_layout()
plt.savefig('hybrid_metrics.png')
plt.show()

In [ ]:
# Display the video
Video(video_file, width=720)

## Hybrid Approach Analysis

### Benefits of the Hybrid Approach:

1. **Exploration and Exploitation Balance**: GA provides broad exploration of the solution space, while PPO offers deep exploitation through gradient-based optimization.

2. **Warm Starting**: Using GA to generate initial policies gives PPO a better starting point, potentially avoiding local optima.

3. **Computational Efficiency**: Instead of running PPO from scratch on random policies, we focus computational resources on promising solutions.

4. **Robustness**: The hybrid approach combines the creativity of evolutionary methods with the precision of reinforcement learning.

### Limitations:

1. **Complexity**: The hybrid approach is more complex to implement and tune.

2. **Computational Cost**: Running both GA and PPO requires more computation than either approach alone.

3. **Architecture Conversion**: Converting between the neural network architectures in GA and PPO requires careful implementation.

4. **Hyperparameter Sensitivity**: The approach introduces additional hyperparameters that need tuning (e.g., when to switch from GA to PPO, how many individuals to fine-tune).